<a href="https://colab.research.google.com/github/megan0422/DABPFinalProject/blob/main/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
!pip install gurobipy
import gurobipy as gp
from gurobipy import *
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from random import randrange
import seaborn as sns
import warnings
import math
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 5.0 MB/s 


In [8]:
# NBI data
url = 'https://raw.githubusercontent.com/megan0422/DABPFinalProject/main/PA22%20-%20PA22.csv'
PA22 = pd.read_csv(url)
# Extract data on allegheny county from the whole dataset
PA22_allegheny  = PA22[PA22["COUNTY_CODE_003"] == 3]
PA22_allegheny  = PA22_allegheny[PA22_allegheny["MAINTENANCE_021"] == 2]
PA22_allegheny

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,BRIDGE_LEN_IND_112,SCOUR_CRITICAL_113,FUTURE_ADT_114,YEAR_OF_FUTURE_ADT_115,MIN_NAV_CLR_MT_116,FED_AGENCY,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA
496,42,1310,1,3,1,885,0,11,3,61000,...,Y,8,13307,2032,0.0,N,42,F,5,15885.73
780,42,2114,1,4,0,0,0,11,3,15216,...,Y,4,16740,2028,0.0,N,42,F,6,835.24
782,42,2120,1,8,0,0,0,11,3,17048,...,Y,8,50,2023,0.0,N,42,F,5,47.95
783,42,2129,1,4,1,0,0,11,3,21024,...,Y,4,1899,2025,0.0,N,42,P,4,81.34
788,42,2135,1,3,0,2020,0,11,3,23000,...,Y,8,4381,2025,0.0,N,42,F,6,2234.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22075,42,54464,1,5,0,0,0,11,3,69584,...,Y,8,5166,2036,0.0,N,42,G,9,264.12
22481,42,56656,1,5,1,0,0,11,3,75816,...,Y,N,18000,2025,NaN,N,42,G,8,476.28
22633,42,57085,1,5,1,0,0,11,3,36808,...,Y,8,550,2042,0.0,N,42,G,7,127.05
22781,42,57402,1,4,2,0,0,11,3,72160,...,Y,8,1124,2038,0.0,N,42,G,8,182.28


In [9]:
PA22_allegheny = PA22_allegheny[["STRUCTURE_NUMBER_008","DETOUR_KILOS_019","ADT_029","STRUCTURE_LEN_MT_049","TOTAL_IMP_COST_096","HIGHWAY_SYSTEM_104","PERCENT_ADT_TRUCK_109"]]
PA22_allegheny.dropna()
PA22_allegheny_cost_nonzero = PA22_allegheny[PA22_allegheny["TOTAL_IMP_COST_096"] != 0]
PA22_allegheny_cost_nonzero.describe()

,DETOUR_KILOS_019,ADT_029,STRUCTURE_LEN_MT_049,TOTAL_IMP_COST_096,HIGHWAY_SYSTEM_104,PERCENT_ADT_TRUCK_109
count,113.000000,113.000000,113.000000,98.000000,113.000000,113.000000
mean,11.681416,5637.407080,64.537168,1943.448980,0.115044,5.176991
std,28.631983,7004.012268,148.252430,12628.895953,0.320497,4.338841
min,0.000000,10.000000,6.100000,41.000000,0.000000,1.000000
25%,3.000000,500.000000,9.400000,72.750000,0.000000,3.000000
50%,5.000000,2889.000000,13.400000,115.500000,0.000000,5.000000
75%,10.000000,8338.000000,21.600000,221.500000,0.000000,5.000000
max,159.000000,35235.000000,947.600000,123526.000000,1.000000,30.000000


In [13]:

i = len(PA22_allegheny_cost_nonzero)
bridge = range(i)

# Step 0. Set up the model
model = Model()

# Step 1. Set up the decision variables
x = model.addVars(bridge, vtype=GRB.BINARY)

addtr = (PA22_allegheny_cost_nonzero["PERCENT_ADT_TRUCK_109"].to_numpy())/100
adt = PA22_allegheny_cost_nonzero["ADT_029"].to_numpy()
hwy = PA22_allegheny_cost_nonzero["HIGHWAY_SYSTEM_104"].to_numpy()
bl = PA22_allegheny_cost_nonzero["STRUCTURE_LEN_MT_049"].to_numpy()
drt = PA22_allegheny_cost_nonzero["DETOUR_KILOS_019"].to_numpy()*1000

#Step 2. Set up the objective function
T = (addtr*adt/24*32*1 + addtr*adt/24*16.2*1.67)*24*365*(drt-bl)
V = (addtr*adt/24*bl*hwy*50*(3.02+0.94) +addtr*adt/24*bl*(1-hwy)*25*(3.02+0.45)+(1-addtr)*adt/24*bl*hwy*50*(3.02+0.45) +(1-addtr)*adt/24*bl*(1-hwy)*25*(3.02+0.45))*24*365*(drt-bl)

model.setObjective (sum((T[i]+V[i])*x[i] for i in bridge))
model.modelSense = GRB.MAXIMIZE

#Step 3. Set up the constraints
#budget constraint
C = PA22_allegheny_cost_nonzero["TOTAL_IMP_COST_096"].to_numpy()*1000
B = 27700000

model.addConstr(sum(C[i]*x[i] for i in bridge) <= B) 
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 113 columns and 98 nonzeros
Model fingerprint: 0x5eb75473
Variable types: 0 continuous, 113 integer (113 binary)
Coefficient statistics:
  Matrix range     [4e+04, 1e+08]
  Objective range  [5e+10, 2e+16]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+07, 3e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 7.066572e+15
Presolve removed 0 rows and 16 columns
Presolve time: 0.00s
Presolved: 1 rows, 97 columns, 97 nonzeros
Variable types: 0 continuous, 97 integer (97 binary)
Found heuristic solution: objective 1.661846e+16

Root relaxation: objective 3.607078e+16, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |

In [14]:
decision_variable = []
for v in model.getVars():
    decision_variable.append(v.x)
print("Total cost is", np.nansum(np.array(decision_variable)*C))

Total cost is 27699000.0


In [15]:
structure_number_with_zero = np.array(decision_variable)*PA22_allegheny_cost_nonzero["STRUCTURE_NUMBER_008"].to_numpy().astype(int)
structure_number = structure_number_with_zero[structure_number_with_zero != 0]
bridge_number = len(structure_number)
print(bridge_number, "bridges should be repair/replacement")
print("The STRUCTURE NUMBER of the bridges to repair/replace are",structure_number)

39 bridges should be repair/replacement
The STRUCTURE NUMBER of the bridges to repair/replace are [ 2114.  2168.  2178.  2179.  2192.  2206.  2250.  2251.  2252.  2282.
  2283.  2334.  2345.  2365.  2383.  2388.  2511.  2526.  2579.  2580.
  2646.  2666.  2714.  2721.  2723. 45729. 46801. 46806. 46944. 47270.
 48142. 48173. 48889. 52733. 54464. 56656. 57085. 57402. 68612.]
